#Réaliser par Foutues pour Foutues avec la technologie GPT3 (OpenAI)

Pour plus d'informations n'hésitez pas a passer sur Twitch : https://twitch.tv/foutu_pour_foutu

Ou a venir sur Twitter : https://twitter.com/vintarpk

# Installation
l'installation n'est nécéssaire que lors de la première execution

In [ ]:
#@title Pip installation
!pip install openai
!pip install ipywidgets
!pip install termcolor

# Paramètres
## Paramètres de base

Pour obtenir votre clé API rendez vous ici :
https://openai.com/api/

Après avoir répondu aux questions de google, cliquer sur votre profil en haut à droite et demander les API key's.
Copier la clé dans la cellule qui va apparaitre à l'éxécutions de la case Connection API


In [ ]:
#@title Import
import openai
import ipywidgets as widgets
from termcolor import colored
from google.colab import files

In [ ]:
#@title Connection API
widPass = widgets.Password(
    value='',
    placeholder='Type something',
    description='API Key',
    disabled=False
)
widPass

In [ ]:
#@title Entrée données


widUsername = widgets.Text(
    value='Username',
    placeholder='Type something',
    description='Name',
    disabled=False
)

widContexte = widgets.Textarea(
    value='un peu de contexte',
    placeholder='Type something',
    description='Contexte',
    disabled=False
)

widAction = widgets.Textarea(
    value='action de départ',
    placeholder='type something',
    disabled=False
)

widBot1Name = widgets.Text(
    value='Nom du bot 1',
    placeholder='Type something',
    description='Bot 1',
    disabled=False
)

widBot1Temp = widgets.FloatSlider(
    value=0.9,
    min=0,
    max=1.0,
    step=0.05,
    description='Temperature',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

widBot1Token = widgets.BoundedIntText(
    value=150,
    min=10,
    max=1500,
    step=1,
    description='max tokens',
    disabled=False
)

widBot2Name = widgets.Text(
    value='Nom du bot 2',
    placeholder='Type something',
    description='Bot 2',
    disabled=False
)

widBot2Temp = widgets.FloatSlider(
    value=0.9,
    min=0,
    max=1.0,
    step=0.05,
    description='Temperature',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

widBot2Token = widgets.BoundedIntText(
    value=150,
    min=10,
    max=1500,
    step=1,
    description='max tokens',
    disabled=False
)


boxContexte = widgets.HBox(
[
    widContexte,
    widAction
])

boxBot1 = widgets.HBox(
[
    widBot1Name,
    widBot1Temp,
    widBot1Token
])

boxBot2 = widgets.HBox(
[
    widBot2Name,
    widBot2Temp,
    widBot2Token
])

center = widgets.VBox(
[
    widUsername,
    boxContexte,
    boxBot1,
    boxBot2,
])

widgets.AppLayout(
    header=widgets.Label("/!\ A modifier avant d'executer la partie initilisation /!\ "),
    center = center
    

)

## Initialisation
Pour redémarrer un nouveau scénario modifier les paramètre ci dessus puis executer le code ci dessous

In [ ]:
#@title Definition de la classe
class GPT:
    #Definition du bot
    def __init__(self,
                 engine='text-davinci-002',
                 temperature=0.5,
                 max_tokens=150,
                 name='bot'):
        self.engine = engine
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.name = name
    
    def parler(self, texte):
        txt = texte + "\n\n" + self.name + " :"
        reponseBrut = self.request(txt)
        reponse = reponseBrut.split('\n')[0]
        return self.name +" : " +reponse
    
    def agir(self, texte):
        txt = texte + "\n\n" + "***" + self.name
        reponseBrut = self.request(txt)
        reponse = "***" + self.name + " " + reponseBrut.split('***')[0] + "***"
        return reponse
    
    def request(self, text):
        response = openai.Completion.create(
          engine=self.engine,
          prompt=text,
          temperature=self.temperature,
          max_tokens=self.max_tokens,
          top_p=1.0,
          frequency_penalty=0.0,
          presence_penalty=0.0
        )

        
        return(response.choices[0].text)

In [ ]:
#@title Definition des fonctions
def faire_conversation(contexte, conversation):
    texte = contexte
    for elt in conversation:
        texte = texte + '\n\n' + elt
    return texte

def send():
    Username = selectUser.value
    input_get = chat.value
    if Username == "evenement":
        ligne = '***' + input_get + '***'
    elif Username == 'contexte':
        ligne = '[' + input_get + ']'
    else:
        ligne = Username + " : " + input_get
    conversation.append(ligne)
    addLabel(ligne, 'humain')
    chat.value = ""

def save(contexte, conversation):
    texte = contexte + "\n\n"
    for elt in conversation:
        texte = texte + "\n\n" + elt
    with open(widSave.value+".txt", "w") as out:
        out.write(texte)
    files.download(widSave.value+".txt")
    print('sauvegarde effectué')
        
def parler_bot(bot, contexte, conversation):
    texte = faire_conversation(contexte, conversation)
    reponse = bot.parler(texte)
    conversation.append(reponse)
    addLabel(reponse, 'bot')

def agir_bot(bot, contexte, conversation):
    texte = faire_conversation(contexte, conversation)
    reponse = bot.agir(texte)
    conversation.append(reponse)
    addLabel(reponse, 'bot')

def addLabel(ligne, typeReponse):
    with MessageBox:
        if typeReponse == 'bot':
            print(ligne)
        elif typeReponse == 'humain':
            print(colored(ligne, 'blue'))

In [ ]:
#@title Recuperation des données rentrées
contexte = "["+ widContexte.value + "] \n\n***" + widAction.value + "***"
global conversation
conversation = []
Username = widUsername.value
bot1 = GPT(name=widBot1Name.value, temperature=widBot1Temp.value, max_tokens=widBot1Token.value, engine="text-davinci-002")
bot2 = GPT(name=widBot2Name.value, temperature=widBot2Temp.value, max_tokens=widBot2Token.value, engine="text-davinci-002")
UserList = [Username, bot1.name, bot2.name, "evenement", "contexte"]
openai.api_key = widPass.value

# Execution
Les fichiers de sortis sont enregistrer dans le dossier ./out/ avec le nom que vous définissez au format txt

In [ ]:
#@title run
style_layout = widgets.Layout(
                align_items='center')



widSave = widgets.Text(
    value='Nom du fichier',
    placeholder='Type something',
    disabled=False
)

parlerBot1 = widgets.Button(
    description=bot1.name + " parler",
    disabled=False,
    button_style='info', 
    tooltip='Click me',
    icon='fa-comment'
)

agirBot1 = widgets.Button(
    description=bot1.name + " agir",
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='fa-hand-paper-o' # (FontAwesome names without the `fa-` prefix)
)

parlerBot2 = widgets.Button(
    description=bot2.name + " parler",
    disabled=False,
    button_style='info', 
    tooltip='Click me',
    icon='fa-comment'
)

agirBot2 = widgets.Button(
    description=bot2.name + " agir",
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='fa-hand-paper-o' # (FontAwesome names without the `fa-` prefix)
)

selectUser = widgets.Dropdown(
    options=UserList,
    value=UserList[0],
    disabled=False,
)

chat = widgets.Text(
    value='',
    placeholder='Type something',
    disabled=False,
    layout=widgets.Layout(width='50%'),
    
)

parler = widgets.Button(
    description="parler",
    disabled=False,
    button_style='info', 
    tooltip='Click me',
    icon='fa-comment'
)

saveButton = widgets.Button(
    description="Download",
    disabled=False,
    button_style='info', 
    tooltip='Click me',
    icon='fa-floppy-o'
)
MessageBox = widgets.Output(layout={'border' : '1px solid black', 'width':'67.5%'})
SaveBox = widgets.HBox([widSave, saveButton])
HeaderBox = widgets.VBox([widgets.Label(contexte), SaveBox])
ActionBot1 = widgets.VBox([parlerBot1, agirBot1])
ActionBot2 = widgets.VBox([parlerBot2, agirBot2])
BotBox = widgets.HBox([MessageBox, ActionBot1, ActionBot2])
ChatBox = widgets.HBox([selectUser, chat, parler])



layout = widgets.AppLayout(
    header=HeaderBox,
    left_sidebar=None,
    right_sidebar=None,
    center = BotBox,
    footer=ChatBox
    

)

display(layout)


def parler_click(b):
    send()

def save_click(b):
    save(contexte, conversation)
    
def bot1_parler_click(b):
    parler_bot(bot1, contexte, conversation)
    
def bot1_agir_click(b):
    agir_bot(bot1, contexte, conversation)
    
def bot2_parler_click(b):
    parler_bot(bot2, contexte, conversation)
    
def bot2_agir_click(b):
    agir_bot(bot2, contexte, conversation)
        
parler.on_click(parler_click)
saveButton.on_click(save_click)
parlerBot1.on_click(bot1_parler_click)
agirBot1.on_click(bot1_agir_click)
parlerBot2.on_click(bot2_parler_click)
agirBot2.on_click(bot2_agir_click)